<a href="https://colab.research.google.com/github/MJSahebnasi/temporal-network-platform/blob/main/thesis_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import networkx as nx

from typing import List, Type
import abc

import copy

import heapq


# **Model**

## Events

In [63]:
class Event:
  def __init__(self, t) -> None:
    self.time = t
    
  def __repr__(self):
    return "<time:" + str(self.time) + ">"

  # will be used in heap, when selecting the earliest event
  def __lt__(self, other):
    return self.time < other.time

  @abc.abstractmethod
  def execute(self, G):
    return

class NodeEvent(Event):
  def __init__(self, t, node) -> None:
    super().__init__(t)
    self.node = node

  def __repr__(self):
    return super().__repr__()[:-1] + "-node:"+str(self.node) + ">"

class NodeEntranceEvent(NodeEvent):
  def __init__(self, t, node, neighbors) -> None:
    super().__init__(t, node)
    self.neighbors = neighbors

  def __repr__(self):
    return super().__repr__()[:-1] + "-entrance" + ">"

  def execute(self, G):
    edges = [(self.node, i) for i in self.neighbors]
    G.add_node(self.node)
    G.add_edges_from(edges)

class NodeDeathEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

  def __repr__(self):
    return super().__repr__()[:-1] + "-death" + ">"

class NodeAttackEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

  def __repr__(self):
    return super().__repr__()[:-1] + "-attack" + ">"

class NodeOnEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

  def __repr__(self):
    return super().__repr__()[:-1] + "-on" + ">"

class NodeOffEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

  def __repr__(self):
    return super().__repr__()[:-1] + "-off" + ">"

## Simulation

In [65]:
class Simulation:   

  ########################## init ##########################

  def create(self, G):
    self.orig_G = G
    self.sim_G = copy.deepcopy(G)

    self.has_entrance_process = False
    self.has_onoff_process = False
    self.has_node_lifespan = False
    self.has_attack_process = False

    self.events = []
    # self.history = []
    # self.snapshots = []

    return self

  def with_entrance_process(self, node_entrance_times):
    ### TODO: parameter validation ###
    self.has_entrance_process = True
    self.node_entrance_times = node_entrance_times
    return self

  def with_node_onoff_process(self, node_on_times, node_off_times):
    ### TODO: parameter validation ###
    self.has_onoff_process = True
    self.node_on_times = node_on_times
    self.node_off_times = node_off_times
    return self

  def with_node_lifespans(self, node_death_times):
    ### TODO: parameter validation ###
    self.has_node_lifespan = True
    self.node_death_times = node_death_times
    return self

  def with_attack_process(self, node_attack_times):
    """
    If you don't want a node to be attacked, just set a NEGATIVE attack time for it.
    example:
    [10, -1, -1, 13]
    The 2nd and the 3rd node will not be attacked.
    """
    ### TODO: parameter validation ###
    self.has_attack_process = True
    self.node_attack_times = node_attack_times
    return self


  ########################## prepair events ##########################


  def create_entrance_events(self):
    n = G.number_of_nodes()
    nodes = list(self.orig_G.nodes())
    self.sim_G = nx.Graph()
    
    ### TODO: later, move to init part
    if len(self.node_entrance_times) != n:
      raise ValueError('!!! len(node_entrance_times) & len(G.nodes) not the same !!!')
    ###

    # dunno how nx handles it, guess set might be faster
    sim_G_nodes = set()
    for i in range(n):
      node = nodes[i]
      sim_G_nodes.add(node)
      neighbors = set.intersection(sim_G_nodes, set(nbr for nbr in self.orig_G.neighbors(i)))
      e = NodeEntranceEvent(self.node_entrance_times[i], node, neighbors)
      self.events.append(e)

  def create_onoff_events(self):
    n = G.number_of_nodes()
    nodes = list(self.orig_G.nodes())
    
    ### TODO: later, move to init part
    if len(self.node_on_times) != n:
      raise ValueError('!!! len(node_on_times) & len(G.nodes) not the same !!!')
    if len(self.node_off_times) != n:
      raise ValueError('!!! len(node_off_times) & len(G.nodes) not the same !!!')
    ###

    for i in range(n):
      node = nodes[i]
      for t in self.node_on_times[i]:
        e = NodeOnEvent(t, node)
        self.events.append(e)
      for t in self.node_off_times[i]:
        e = NodeOffEvent(t, node)
        self.events.append(e)

  def create_death_events(self):
    n = G.number_of_nodes()
    nodes = list(self.orig_G.nodes())
    
    ### TODO: later, move to init part
    if len(self.node_death_times) != n:
      raise ValueError('!!! len(node_death_times) & len(G.nodes) not the same !!!')
    ###

    for i in range(n):
      node = nodes[i]
      e = NodeDeathEvent(self.node_death_times[i], node)
      self.events.append(e)

  def create_attack_events(self):
    n = G.number_of_nodes()
    nodes = list(self.orig_G.nodes())
    
    ### TODO: later, move to init part
    if len(self.node_attack_times) != n:
      raise ValueError('!!! len(node_attack_times) & len(G.nodes) not the same !!!')
    ###

    for i in range(n):
      node = nodes[i]
      t = self.node_attack_times[i]

      if t < 0:
        continue

      e = NodeAttackEvent(t, node)
      self.events.append(e)

  def prepair_events(self):
    if self.has_entrance_process:
      self.create_entrance_events()

    if self.has_onoff_process:
      self.create_onoff_events()

    if self.has_node_lifespan:
      self.create_death_events()

    if self.has_attack_process:
      self.create_attack_events()

    heapq.heapify(self.events)


  ########################## run ##########################


  def run(self):
    self.prepair_events()
    sim_time = 0
    
    ###
    # print(self.events)
    ###

    while len(self.events) > 0:
      e = heapq.heappop(self.events)
      e.execute(self.sim_G)
      # sim_time = e.time

      # handle history/snapshot
      # pass

    print(self.sim_G)
    print(self.sim_G.nodes())
    print(self.sim_G.edges())

In [71]:
G = nx.erdos_renyi_graph(n=3,p=0.6)
ent = [0, 1, 2]
dth = [10, 50, 55]
atk = [8, -1, -1]
off = [[3], [10, 20], []]
on  = [[5], [12], []]

print(G)
print(G.nodes())
print(G.edges())

s = Simulation().create(G).with_entrance_process(ent).with_node_lifespans(dth).with_attack_process(atk).with_node_onoff_process(on, off).run()

Graph with 3 nodes and 3 edges
[0, 1, 2]
[(0, 1), (0, 2), (1, 2)]
Graph with 3 nodes and 3 edges
[0, 1, 2]
[(0, 1), (0, 2), (1, 2)]
